In [20]:
# Libraries
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Library
import time

In [23]:

# Url of the website to scrape
url = "https://lodzka.policja.gov.pl/ld/form/63,Statystyki-dnia.html?page=0"
# List to store scraped values
values = list()
# Initialization of page index
i = 0


In [ ]:
# Error handling try-except block to prevent from index error
try:
    # Loop statement: we loop through all url's that contain word "Statystyki"
    while "Statystyki" in url:
        # Send respond to the server with specified url
        response = requests.get(url = url)
        # Assert whether the status of response is 200 which means: Successful response
        assert response.status_code == 200
        # Get html content of the website
        html = response.text
        # Parse the html content
        soup = BeautifulSoup(html, "html.parser")
        # Extract the headers of the tables (they are stored in: table tag -> attribute class with value table-listing margin_b20 -> tag th)
        headers = [header.text for header in soup.findAll("table", {"class":"table-listing margin_b20"})[0].find_all({"th"})]
        # Extend the values list about scraped values that are in format of comprehensive list
        values.extend([value.text for value in soup.findAll("table", {"class":"table-listing margin_b20"})[0].find_all({"td"})])
        # Find all hypertext references (they were stored in: div tag -> attribute id with value meni_strony -> tag a)
        pages_linked = [page.get("href") for page in soup.findAll("div", {"id":"meni_strony"})[0].findAll("a")]
        # As there are many hyperlinks for every page, we choose the page that has index about one higher that one we are on
        pages_next = list(filter(lambda x: x if "page="+str(i+1) in x else 0, pages_linked))[0]
        # Assign new url to url variable
        url = url.split("/ld/")[0]+pages_next
        # Increment the page index
        i += 1
        # We use time sleep with gamma distribution (right-skewed distribution) to prevent being recognized as a bot
        time.sleep(np.ceil(np.random.gamma(2., 2.)))
# When the IndexError occurs, we looped through all websites
except IndexError:
    print("All sites has been scraped")



In [36]:
# List of lists that will be then converted to rows in pandas dataframe
observations = list()
# Append list of observations for every day to observations list
for i in range(int(len(values) / 7)):
    observations.append(values[7*i:7*(i+1)])

In [60]:
# Create pandas dataframe
data = pd.DataFrame(observations)
# Change headers to scraped ones
data.columns = headers
# Convert column "Data" to datetime
data["Data"] = data["Data"].astype("datetime64[ns]")
# Create a function that converts all object columns to numeric data type
def object_to_integer(dataframe):
    """Converts object columns to integer data type"""
    for column in dataframe.columns:
        if dataframe[column].dtype == "object":
            dataframe[column] = dataframe[column].astype("int")
object_to_integer(data)
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2954 entries, 0 to 2953
Data columns (total 7 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   Data                             2954 non-null   datetime64[ns]
 1   Zatrzymani na gorącym uczynku    2954 non-null   int64         
 2   Zatrzymani poszukiwani           2954 non-null   int64         
 3   Zatrzymani nietrzeźwi kierujący  2954 non-null   int64         
 4   Wypadki drogowe                  2954 non-null   int64         
 5   Zabici w wypadkach               2954 non-null   int64         
 6   Ranni w wypadkach                2954 non-null   int64         
dtypes: datetime64[ns](1), int64(6)
memory usage: 161.7 KB
